In [3]:
import pandas as pd
import networkx as nx


In [4]:
# load tsv file of all convergent clusters
convergent_clones = pd.read_csv('/mnt/susanne/datasets/convergence/clusters.tsv', sep='\t')



In [5]:
# get the list of distinct cluster ids
cluster_ids = convergent_clones['unique_convergent_clone_id'].unique()
# group by convergent_clones by sequence_id and keep only those with more than one cluster
convergent_clones = convergent_clones.groupby('sequence_id').filter(lambda x: len(x) > 1)
# group convergent clones by sequence_id and for each group, get the set of unique_convergent_clone_id's and add them to a list
lists = convergent_clones.groupby('sequence_id')['unique_convergent_clone_id'].unique()
# get the unique lists
lists = lists.apply(tuple)
unique_lists = set(lists)


In [6]:
# convert the tuples to lists and the set to a list
unique_lists = list(unique_lists)
unique_lists = [list(x) for x in unique_lists]

# create a graph with networkx
G = nx.Graph()
# add cluster ids as nodes
G.add_nodes_from(cluster_ids)
# add edges between the clusters in each row in shared_seqs (clone_list)
for i, clone_list in enumerate(unique_lists):
	for i in range(len(clone_list)):
		for j in range(i+1, len(clone_list)):
			G.add_edge(clone_list[i], clone_list[j])

# get some basic information about the graph
print(G.number_of_nodes())
print(G.number_of_edges())

1045739
116455


In [8]:
# get the connected components of the graph
connected_components = list(nx.connected_components(G))
# make each list of connected components a set and convert it to a comma separated string
connected_components = [','.join(map(str, x)) for x in connected_components]
# save the connected components to a file using pandas
df = pd.DataFrame({'connected_components': connected_components})
df.to_csv('/mnt/susanne/datasets/convergence/connected_components.tsv', sep='\t', index=False, header=False)